In [1]:
import geotable
import pandas as pd

/home/harmon/anaconda3/envs/pygmt/lib/python3.12/site-packages/osgeo/osr.py:410: FutureWarning: Neither osr.UseExceptions() nor osr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [3]:
t  = geotable.load('B4_tiled_laz_NW/B4_index.kmz')
obj = t['geometry_object']

In [4]:
t

,Name,geometry_object,geometry_layer,geometry_proj4
0,b4_172000_3994000,POLYGON Z ((-120.6184063379407 36.046308771231...,B4_index,+proj=longlat +datum=WGS84 +no_defs
1,b4_172000_3996000,POLYGON Z ((-120.6187560282552 36.053938941356...,B4_index,+proj=longlat +datum=WGS84 +no_defs
2,b4_174000_3992000,POLYGON Z ((-120.5953561841213 36.027414535833...,B4_index,+proj=longlat +datum=WGS84 +no_defs
3,b4_174000_3994000,POLYGON Z ((-120.5957746721468 36.036610591814...,B4_index,+proj=longlat +datum=WGS84 +no_defs
4,b4_174000_3996000,POLYGON Z ((-120.6073427674509 36.054283365092...,B4_index,+proj=longlat +datum=WGS84 +no_defs
...,...,...,...,...
1035,b4_624000_3638000,POLYGON Z ((-115.6529327965162 32.884966301155...,B4_index,+proj=longlat +datum=WGS84 +no_defs
1036,b4_624000_3640000,POLYGON Z ((-115.6528432652535 32.890882134659...,B4_index,+proj=longlat +datum=WGS84 +no_defs
1037,b4_624000_3642000,POLYGON Z ((-115.661892614875 32.9090183219207...,B4_index,+proj=longlat +datum=WGS84 +no_defs
1038,b4_626000_3638000,POLYGON Z ((-115.6395039611111 32.879535489656...,B4_index,+proj=longlat +datum=WGS84 +no_defs


In [5]:
def extract_points(this_obj):
    points = list(this_obj.exterior.coords) 
    p1 = points[0][0:2]
    p2= points[1][0:2]
    p3 = points[2][0:2]
    p4 = points[3][0:2]
    return p1, p2, p3,p4

In [6]:
def point_inside_quadrilateral(p1,p2,p3,p4,p):
    # Determine the bounding box of the quadrilateral
    x1 = p1[0]
    x2 = p2[0]
    x3 = p3[0]
    x4 = p4[0]
    y1 = p1[1]
    y2 = p2[1]
    y3 = p3[1]
    y4 = p4[1]
    x = p[0]
    y = p[1]
    min_x = min(x1, x2, x3, x4)
    max_x = max(x1, x2, x3, x4)
    min_y = min(y1, y2, y3, y4)
    max_y = max(y1, y2, y3, y4)
    
    # Check if the point is inside the bounding box
    if x < min_x or x > max_x or y < min_y or y > max_y:
        return False
    
    # Count the intersections of the point with the quadrilateral edges
    intersections = 0
    
    # Check each edge of the quadrilateral
    edges = [(x1, y1, x2, y2), (x2, y2, x3, y3), (x3, y3, x4, y4), (x4, y4, x1, y1)]
    for edge in edges:
        x1_edge, y1_edge, x2_edge, y2_edge = edge
        # Check if the point is on the same side of the edge's line
        # as the vertex with the smaller y-coordinate
        if (y1_edge > y) != (y2_edge > y):
            # Calculate the x-coordinate of the intersection point
            intersection_x = (x2_edge - x1_edge) * (y - y1_edge) / (y2_edge - y1_edge) + x1_edge
            # Check if the intersection point is to the right of the point
            if x < intersection_x:
                intersections += 1
    
    # If the number of intersections is odd, the point is inside the quadrilateral
    return intersections % 2 == 1


In [7]:
#p = (-116.593315,33.537045)

In [8]:
stalistname_all = 'stations_info_all.xlsx'
stainfo = pd.read_excel(stalistname_all)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 

In [11]:
files = set()   
for j in range(len(stalist_all)):
    p = (lon_stations_all[j],lat_stations_all[j])
    for i in range(len(obj)):
        obj_this = obj[i]
        p1,p2,p3,p4 = extract_points(obj_this)
        if point_inside_quadrilateral(p1,p2,p3,p4,p):
            files.add(t['Name'][i])

In [12]:
files

{'b4_536000_3710000', 'b4_538000_3710000'}